In [96]:
%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import importlib

try:
    importlib.reload(dda)
except NameError:
    import dd_analysis as dda

In [3]:
# Load data from previous runs
res_paths = {
    'Original': 'data/dd_res_2020-12-03_01-31-32.npz',
    'Merged repr': 'data/merged_repr_dd_res_2020-12-04_05-15-35.npz',
#    'Small item repr': 'data/small_item_repr_dd_res_2020-11-24_18-48-00.npz',
#    'More compressed': 'data/all_ratios_0.5_dd_res_2020-11-24_21-17-07.npz',
#    'Half-size HL': 'data/half_hidden_longer_dd_res_2020-11-25_01-06-37.npz',
    'No item repr': 'data/no_item_repr_reallocate_dd_res_2020-12-04_06-30-32.npz',
    'No context repr': 'data/no_ctx_repr_reallocate_dd_res_2020-12-04_07-48-18.npz',
    'No repr': 'data/no_repr_reallocate_dd_res_2020-12-03_03-24-49.npz',
#     'Short original': 'data/short_save_params_dd_res_2020-11-26_11-44-16.npz',
#     'Short no item repr': 'data/short_save_params_no_item_repr_dd_res_2020-11-26_11-58-59.npz',
#     'Short no context repr': 'data/short_save_params_no_ctx_repr_dd_res_2020-11-26_12-05-47.npz'
}

res_data = {name: dda.get_result_means(path) for name, path in res_paths.items()}

C:\Users\ethan\anaconda3\envs\cuda11\lib\site-packages\scipy\stats\_distn_infrastructure.py:2023: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\ethan\anaconda3\envs\cuda11\lib\site-packages\scipy\stats\_distn_infrastructure.py:2024: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [4]:
# Get ready to look at some particular runs
curr_runs = ['Original', 'Merged repr', 'No item repr', 'No context repr', 'No repr']
curr_sets = {name: data for name, data in res_data.items() if name in curr_runs}
n_curr = len(curr_runs)

# short_runs = ['Short original', 'Short no item repr', 'Short no context repr']
# short_sets = {name: data for name, data in res_data.items() if name in short_runs}

plt.close('all')

In [19]:
# Plot loss, accuracy, and test accuracy
fig, axs = dda.make_plot_grid(3, 1, ax_dims=(7, 3), ravel=True)

for (label, res), col in zip(curr_sets.items(), mcolors.TABLEAU_COLORS):
    dda.plot_report(axs[0], res, 'loss', label=label)
    dda.plot_report(axs[1], res, 'weighted_acc', label=label)
    dda.plot_report(axs[2], res, 'test_weighted_acc', label=label)

for ax in axs:
    ax.legend()

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# Item RSA matrices
inds_to_plot = [0, 1, 4, 11, 12, 30, 50, -1]
fig, axss = dda.auto_subplots(len(inds_to_plot), n_curr, ax_dims=(3, 3))
fig.suptitle('Mean RSA for items (hidden layer)')

for ind, axs in zip(inds_to_plot, axss):
    for (label, res), ax in zip(curr_sets.items(), axs):
        dda.plot_rsa(ax, res, 'item_hidden', ind, title_addon=label)#, item_order='group-outer')

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# Corresponding dendrograms
fig, axss = dda.auto_subplots(len(inds_to_plot), n_curr, ax_dims=(4, 3))
fig.suptitle('Item representation similarity')

for ind, axs in zip(inds_to_plot, axss):
    for (label, res), ax in zip(curr_sets.items(), axs):
        dda.plot_repr_dendrogram(ax, res, 'item_hidden', ind, title_addon=label)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
# Look at RSA projections onto group, domain, and item models
# First plot the models
item_models = dda.make_ortho_item_rsa_models(**next(iter(curr_sets.values()))['net_params'])

fig, axs = dda.auto_subplots(2, (len(item_models)+1)//2, ax_dims=(3.4, 3.4))
for ax, (mtype, model) in zip(axs.ravel(), item_models.items()):
    dda.plot_rsa_model(ax, model, input_type='item')
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontsize('x-small') 
    ax.set_title(mtype + ' model RDM')
    
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
# Item hidden layer projections
fig, axs = dda.make_plot_grid(len(item_models), 1, ax_dims=(9, 3), ravel=True)

snap_type = 'item_hidden'

for (label, res), col in zip(curr_sets.items(), mcolors.TABLEAU_COLORS):
    dda.plot_rdm_projections(res, snap_type, axs, color=col, label=label, normalize=True)
    
for ax in axs:
    dda.outside_legend(ax)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
# Item representation layer projections
fig, axs = dda.make_plot_grid(len(item_models), 1, ax_dims=(9, 3), ravel=True)

snap_type = 'item'

for (label, res), col in zip(curr_sets.items(), mcolors.TABLEAU_COLORS):
    # only plot if we actually have an item representation layer
    if 'item' in res['repr_dists']:
        dda.plot_rdm_projections(res, snap_type, axs, color=col, label=label, normalize=True)
    
for ax in axs:
    dda.outside_legend(ax)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
# Context hidden layer projections
# First plot models
ctx_models = dda.make_ortho_context_rsa_models(**next(iter(curr_sets.values()))['net_params'])

fig, axs = dda.auto_subplots(1, len(ctx_models), ax_dims=(3.4, 3.4))
for ax, (mtype, model) in zip(axs.ravel(), ctx_models.items()):
    dda.plot_rsa_model(ax, model, input_type='context')
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontsize('x-small') 
    ax.set_title(mtype + ' model RDM')
    
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
fig, axs = dda.make_plot_grid(2, 1, ax_dims=(9, 3), ravel=True)

snap_type = 'context_hidden'

for (label, res), col in zip(curr_sets.items(), mcolors.TABLEAU_COLORS):
    dda.plot_rdm_projections(res, snap_type, axs, color=col, label=label)
    
for ax in axs:
    dda.outside_legend(ax)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:
# Context representation layer projections
fig, axs = dda.make_plot_grid(2, 1, ax_dims=(9, 3), ravel=True)

snap_type = 'context'

for (label, res), col in zip(curr_sets.items(), mcolors.TABLEAU_COLORS):
    # only plot if we actually have a context representation layer
    if 'context' in res['repr_dists']:
        dda.plot_rdm_projections(res, snap_type, axs, color=col, label=label)
    
for ax in axs:
    dda.outside_legend(ax)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
# All layers item projection
fig, axs = dda.make_plot_grid(4, 1, ax_dims=(9, 3), ravel=True)

snap_type = 'item_full'

for (label, res), col in zip(curr_sets.items(), mcolors.TABLEAU_COLORS):
    dda.plot_rdm_projections(res, snap_type, axs, color=col, label=label, normalize=True)
    
for ax in axs:
    dda.outside_legend(ax)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
# All layers context projection
fig, axs = dda.make_plot_grid(2, 1, ax_dims=(9, 3), ravel=True)

snap_type = 'context_full'

for (label, res), col in zip(curr_sets.items(), mcolors.TABLEAU_COLORS):
    dda.plot_rdm_projections(res, snap_type, axs, color=col, label=label)
    
for ax in axs:
    dda.outside_legend(ax)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [100]:
# Fit linear models
reg_data = dda.make_dict_for_regression(curr_sets.values())

In [94]:
# Fit model for context domain information interacting with item domain and attribute similarity (both potentially important)
model1_res = dda.fit_linear_model('test_weighted_acc ~ ctx_same_vs_different_domain*(item_attribute_similarity + item_same_vs_different_domain)', reg_data)
print(model1_res.summary())

                            OLS Regression Results                            
Dep. Variable:      test_weighted_acc   R-squared:                       0.622
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     5971.
Date:                Tue, 08 Dec 2020   Prob (F-statistic):               0.00
Time:                        17:46:18   Log-Likelihood:                 21863.
No. Observations:               18180   AIC:                        -4.371e+04
Df Residuals:                   18174   BIC:                        -4.367e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [101]:
model2_res = dda.fit_linear_model('test_weighted_acc ~ item_attribute_similarity + ctx_same_vs_different_domain', reg_data)
print(model2_res.summary())

                            OLS Regression Results                            
Dep. Variable:      test_weighted_acc   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                     7451.
Date:                Tue, 08 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:36:17   Log-Likelihood:                 18472.
No. Observations:               18180   AIC:                        -3.694e+04
Df Residuals:                   18177   BIC:                        -3.691e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [ ]:
# TODO: Try plotting a combination of item attribute similarity and context domain measures? Not sure if it makes sense.

In [26]:
# Context RSA matrix
inds_to_plot = [0, 1, 4, 11, 12, 30, 50, -1]
fig, axss = dda.auto_subplots(len(inds_to_plot), n_curr, ax_dims=(4, 4))
fig.suptitle('Mean RSA for contexts')

#ctx_sets = {name: res for name, res in curr_sets.items() if name in ['Original', 'Merged repr', 'No item repr']}

for ind, axs in zip(inds_to_plot, axss):
    for (label, res), ax in zip(curr_sets.items(), axs):
        dda.plot_rsa(ax, res, 'context_hidden', ind, title_addon=label, item_order='domain-inner')

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
# Corresponding context dendrograms
fig, axss = dda.auto_subplots(len(inds_to_plot), n_curr, ax_dims=(4, 3))
fig.suptitle('Context representation similarity')

for ind, axs in zip(inds_to_plot, axss):
    for (label, res), ax in zip(curr_sets.items(), axs):
        dda.plot_repr_dendrogram(ax, res, 'context', ind, title_addon=label)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
# Item rep RSA over all snapshots (useful maybe for finding interesting change points)
fig, axs = dda.make_plot_grid(len(curr_sets), 2, ax_dims=(5, 5))

for ax, (label, res) in zip(axs, curr_sets.items()):
    n_snaps = len(res['snap_epochs'])
    snap_freq = res['train_params']['snap_freq']
    im = ax.imshow(res['repr_dists']['item']['all'],
                   extent=(-0.5, n_snaps * snap_freq-0.5, n_snaps * snap_freq-0.5, -0.5))
    ax.set_xticks(res['snap_epochs'][::4])
    ax.tick_params(axis='x', labelrotation=45)
    ax.set_yticks(res['snap_epochs'][::4])
    ax.set_xlabel('Epochs')
    ax.set_title(f'Item repr distances over training ({label})')
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
# MDS of item representations over time
for label, res in curr_sets.items():
    dda.plot_repr_trajectories(res, 'item', dims=3, title_label=label)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'item'

In [81]:
for label, res in curr_sets.items():
    dda.plot_repr_trajectories(res, 'context', dims=2, title_label=label)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
import importlib
importlib.reload(dda)

fig, axs = dda.make_plot_grid(len(curr_runs), 2, ax_dims=(6, 15))
run_num = 0
snap_index = 5

for (name, res), ax in zip(short_sets.items(), axs.ravel()):
    dda.plot_hl_input_pattern_correlations(ax, res, run_num, snap_index, title_label=name)
    
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …